In [1]:
import os
import time
from utils import green_print

In [2]:
# Optional
!rm -rf ~/.clap/storage/*.db 

In [3]:
green_print('Listing providers file contents:')
!cat ~/.clap/configs/providers.yaml
print('-------\n')

green_print('Listing logins file contents:')
!cat ~/.clap/configs/logins.yaml
print('-------\n')

green_print('Listing instances file contents:')
!cat ~/.clap/configs/instances.yaml
print('-------\n')

green_print("Listing CLAP's instance templates:")
!clapp node list-templates
print('-------\n')

Listing providers file contents:
aws-config-us-east-1:
    provider: aws
    access_keyfile: ec2_access_key.pub
    secret_access_keyfile: ec2_access_key.pem
    url: https://ec2.us-east-1.amazonaws.com
    region: us-east-1
-------

Listing logins file contents:
login-ubuntu:
    user: ubuntu
    keypair_name: otavio_key_us_east_1
    keypair_public_file: otavio_key_us_east_1.pub
    keypair_private_file: otavio_key_us_east_1.pem
    sudo: true
    sudo_user: root
    
login-ubuntu-b:
    user: ubuntu
-------

Listing instances file contents:
type-a:
    provider: aws-config-us-east-1
    login: login-ubuntu
    flavor: t2.micro
    image_id: ami-07d0cf3af28718ef8
    security_group: otavio-sg

type-b:
    provider: aws-config-us-east-1
    login: login-ubuntu-b
    flavor: t2.medium
    image_id: ami-07d0cf3af28718ef8
    boot_disk_size: 10
-------

Listing CLAP's instance templates:
name: `type-a`
    provider config id:` aws-config-us-east-1`
    login config id: `login-ubuntu`

na

In [4]:
description = 'example-mpi-cluster'

!clapp mpi create-mcluster type-a:2 --desc {description}





PLAY [localhost] ***************************************************************

TASK [Starting 2 type-a instances (timeout 600 seconds)] ***********************
changed: [localhost]












PLAY [localhost] ***************************************************************

TASK [ec2_tag] *****************************************************************

changed: [localhost] => (item={'id': 'i-0936eabc88c75d90e', 'name': 'aws-config-us-east-1-node-0-JamesRuggles'})
changed: [localhost] => (item={'id': 'i-0d82afa07630f9595', 'name': 'aws-config-us-east-1-node-1-DavidCorrie'})




Cluster `mcluster-0` successfully created!


In [5]:
!clapp mpi list-mclusters

* MPI cluster of id `mcluster-0` associated with cluster `cluster-0`. Status: RUNNING. Coordinator id is: node-0, slaves are: ['node-1']. Jobs configurated are: []
Listed 1 MPI clusters


In [6]:
cluster_id = 'mcluster-0'
efs_mount_point = '172.33.86.13'

!clapp mpi setup-mcluster {cluster_id} --mount_ip {efs_mount_point}


PLAY [EC2 EFS Playbook] ********************************************************



TASK [Gathering Facts] *********************************************************
ok: [node-1]ok: [node-1]
ok: [node-0]

TASK [include_role : nfs-client] ***********************************************

TASK [nfs-client : Perform package list update] ********************************
changed: [node-1]
changed: [node-0]

TASK [nfs-client : install NFS client software (Debian/Ubuntu)] ****************

changed: [node-1]
changed: [node-0]

TASK [nfs-client : install NFS client software (RHEL-compatible)] **************
skipping: [node-0]
skipping: [node-1]

TASK [nfs-client : Ensure `rpcbind` is running (Debian)] ***********************
skipping: [node-0]
skipping: [node-1]

TASK [nfs-client : Ensure `rpcbind` is running (RHEL-compatible)] **************
skipping: [node-0]
skipping: [node-1]

TASK [nfs-client : Ensure `portmap` is running (Ubuntu prior to 14.04)] ********
skipping: [node-0]
skipping: [node-

In [7]:
green_print('Get NPB files to local folder')
!mkdir -p NPB
!wget https://www.nas.nasa.gov/assets/npb/NPB3.4.1.tar.gz -O ./NPB/NPB3.4.1.tar.gz 

green_print('Extract NPB files to local folder')
!tar -C NPB -xvf ./NPB/NPB3.4.1.tar.gz

green_print('Create a make configuration')
!cp NPB/NPB3.4.1/NPB3.4-MPI/config/make.def.template NPB/NPB3.4.1/NPB3.4-MPI/config/make.def 

green_print('All done')

Get NPB files to local folder
--2021-03-28 01:24:51--  https://www.nas.nasa.gov/assets/npb/NPB3.4.1.tar.gz
Resolving www.nas.nasa.gov (www.nas.nasa.gov)... 2001:4d0:6318:903:198:9:3:30, 198.9.3.30
Connecting to www.nas.nasa.gov (www.nas.nasa.gov)|2001:4d0:6318:903:198:9:3:30|:443... failed: No route to host.
Connecting to www.nas.nasa.gov (www.nas.nasa.gov)|198.9.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 428294 (418K) [application/x-gzip]
Saving to: ‘./NPB/NPB3.4.1.tar.gz’

./NPB/NPB3.4.1.tar. 100%[===================>] 418,26K   488KB/s    in 0,9s    

2021-03-28 01:24:56 (488 KB/s) - ‘./NPB/NPB3.4.1.tar.gz’ saved [428294/428294]

Extract NPB files to local folder
NPB3.4.1/
NPB3.4.1/NPB3.4-OMP/
NPB3.4.1/NPB3.4-OMP/SP/
NPB3.4.1/NPB3.4-OMP/SP/z_solve.f90
NPB3.4.1/NPB3.4-OMP/SP/blk_par0.h
NPB3.4.1/NPB3.4-OMP/SP/set_constants.f90
NPB3.4.1/NPB3.4-OMP/SP/error.f90
NPB3.4.1/NPB3.4-OMP/SP/exact_rhs.f90
NPB3.4.1/NPB3.4-OMP/SP/Makefile
NPB3.4.1/NPB3.4-OMP/SP

In [8]:
job_alias = 'example-mpi-job'
green_print('Creating a MPI job')
!clapp mpi create-job {cluster_id} --job_name {job_alias}

Creating a MPI job

PLAY [coordinator] *************************************************************

TASK [Gathering Facts] *********************************************************














ok: [node-0]

TASK [Creates directory] *******************************************************
changed: [node-0]

TASK [Create meta info file] ***************************************************
changed: [node-0]

TASK [Creates directory which will contain tasks infos] ************************
changed: [node-0]

TASK [Save job and cluster info in the info file (for data processing purposes)] ***
changed: [node-0]

TASK [Remove job folder if it failed] ******************************************
skipping: [node-0]



TASK [Fail job creation if creation not successful] ****************************
skipping: [node-0]








Job `job-0` successfully created
Job: job-0 at mcluster-0 (alias: example-mpi-job) in path: `/home/ubuntu/efs/mcluster-0/job-0`


In [9]:
green_print('List jobs')
!clapp mpi list-jobs

List jobs
Job: job-0 at mcluster-0 (alias: example-mpi-job) in path: `/home/ubuntu/efs/mcluster-0/job-0`
Listed 1 jobs


In [10]:
job_id = 'job-0'
directory_to_push = os.path.abspath('./NPB/NPB3.4.1')

green_print(f'Adding files from {directory_to_push} to job {job_id}')
!clapp mpi push-job-files {job_id} {directory_to_push}

Adding files from /home/lopani/Projects/clap_imp/examples/NPB/NPB3.4.1 to job job-0

PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************














ok: [node-0]

TASK [create the execution path recursively if the same does not exist] ********
ok: [node-0]

TASK [passing file] ************************************************************
changed: [node-0]










In [11]:
compile_script = '''#!/bin/bash
make -C NPB3.4.1/NPB3.4-MPI is CLASS=C
'''

compile_script_path = os.path.abspath('./NPB/compile.sh')

with open(compile_script_path, 'wt') as f:
    f.write(compile_script)
    print(f'Compile script saved to {compile_script_path}')

!chmod +x {compile_script_path}

green_print(f'Setting up job {job_id}')
!clapp mpi setup-job {job_id} {compile_script_path}


Compile script saved to /home/lopani/Projects/clap_imp/examples/NPB/compile.sh
Setting up job job-0

PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************







ok: [node-0]

TASK [create the execution path recursively if the same does not exist] ********
ok: [node-0]

TASK [Make time permanent (simply setting as variable will make the time to be reevaluated every time)] ***
ok: [node-0]ok: [node-0]

TASK [passing file] ************************************************************
changed: [node-0]

TASK [Compile script (up to two hours)] ****************************************
changed: [node-0]

TASK [Keep probing compiling progression] **************************************
changed: [node-0]

TASK [Remove file (delete file)] ***********************************************
changed: [node-0]










In [12]:
green_print('Generatig hostfile')

!clapp mpi gen-job-hostfile {job_id}

Generatig hostfile

PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************














ok: [node-0]







ok: [node-1]

TASK [Get number of cpus of each host] *****************************************
changed: [node-1]
changed: [node-0]

PLAY [coordinator] *************************************************************

TASK [Gathering Facts] *********************************************************
ok: [node-0]ok: [node-0]

TASK [Remove hosts file] *******************************************************
ok: [node-0]

TASK [Create hosts file] *******************************************************

changed: [node-0]

TASK [Add Ansible inventory mappings to /home/ubuntu/efs/mcluster-0/job-0//hosts] ***
skipping: [node-0]


TASK [Add Ansible inventory mappings to /home/ubuntu/efs/mcluster-0/job-0//hosts] ***

changed: [node-0]







In [13]:
execution_script = '''#!/bin/bash
mpirun --hostfile hosts NPB3.4.1/NPB3.4-MPI/bin/is.C.x
'''

execution_script_path = os.path.abspath('./NPB/run.sh')

with open(execution_script_path, 'wt') as f:
    f.write(execution_script)
    !chmod +x {execution_script_path}
    print(f'Run script saved to {execution_script_path}')

green_print(f'Running job {job_id}')
!clapp mpi run-job-task {job_id} {execution_script_path}

Run script saved to /home/lopani/Projects/clap_imp/examples/NPB/run.sh
Running job job-0

PLAY [coordinator] *************************************************************


TASK [Gathering Facts] *********************************************************














ok: [node-0]

TASK [passing file] ************************************************************
changed: [node-0]



TASK [Make time permanent (simply setting as variable will make the time to be reevaluated every time)] ***
ok: [node-0]

TASK [Setting CLAPP_MPI_CLUSTER, CLAPP_MPI_JOB CLAPP_MPI_TASK] *****************
ok: [node-0]

TASK [Generate output files] ***************************************************
changed: [node-0]

TASK [Set execution_dir as   CLAPP_MPI_JOB_DIR if run_on_taskdir is set to true] ***
ok: [node-0]

TASK [Append execution specific info] ******************************************

changed: [node-0]

TASK [Run application] *********************************************************
changed: [node-0]


In [14]:
green_print('waiting a while...')
time.sleep(30)

waiting a while...


In [15]:
results_folder = os.path.abspath('./NPB/results')
!mkdir -p {results_folder}
!clapp mpi fetch-tasks-results {job_id} {results_folder}
green_print(f'Results from job {job_id} copied to {results_folder}')


PLAY [coordinator] *************************************************************

TASK [Create job specific folder] **********************************************

changed: [node-0]

TASK [Fetch tasks info (.out and .err)] ****************************************
changed: [node-0]changed: [node-0]




Job job-0 fetched to destination /home/lopani/Projects/clap_imp/examples/NPB/results
Results from job job-0 copied to /home/lopani/Projects/clap_imp/examples/NPB/results


In [16]:
green_print('Results')
!cat {results_folder}/{job_id}/*/*.out 

Results
--------------------------------------------------------------------------
[[39738,1],0]: A high-performance Open MPI point-to-point messaging module
was unable to find any relevant network interfaces:

Module: OpenFabrics (openib)
  Host: ip-172-31-73-0

Another transport will be used instead, although this may result in
lower performance.

NOTE: You can disable this warning by setting the MCA parameter
btl_base_warn_component_unused to 0.
--------------------------------------------------------------------------


 NAS Parallel Benchmarks 3.4 -- IS Benchmark

 Size:  134217728  (class C)
 Iterations:   10
 Total number of processes:  2

   iteration
        1
        2
        3
        4
        5
        6
        7
        8
        9
        10


 IS Benchmark Completed
 Class           =                        C
 Size            =                134217728
 Iterations      =                       10
 Time in seconds =                    89.85
 Total processes =           

In [17]:
green_print(f'Destroying cluster {cluster_id}')
!clapp mpi destroy-mcluster {cluster_id}

Destroying cluster mcluster-0

PLAY [coordinator] *************************************************************

TASK [Gathering Facts] *********************************************************














ok: [node-0]

TASK [Deleting the mpc directory] **********************************************
skipping: [node-0]

PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [node-0]

TASK [Mount EFS volume] ********************************************************

ok: [node-0]











PLAY [localhost] ***************************************************************

TASK [Stopping nodes `node-0, node-1`] *****************************************
changed: [localhost]




Cluster `mcluster-0` destroyed


In [18]:
!clapp mpi list-mclusters

* MPI cluster of id `mcluster-0` associated with cluster `cluster-0`. Status: TERMINATED. Coordinator id is: node-0, slaves are: ['node-1']. Jobs configurated are: ['job-0']
Listed 1 MPI clusters


In [19]:
!rm -rf NPB